# 广播机制

In [1]:
import numpy as np
import colorama as co

from display import aprint
from array_ import arange_by_shape

## 1. 广播机制说明

Numpy 提供了广播 (broadcast) 机制, 用于对不同形状的数组进行数学操作的功能, 对于 `x` 和 `y` 两个矩阵:

$x = \begin{bmatrix}
    \begin{bmatrix}
        -0.0 & -0.1 & -0.2 & -0.3
    \end{bmatrix} \\
    \begin{bmatrix}
        -0.4 & -0.5 & -0.6 & -0.7
    \end{bmatrix} \\
    \begin{bmatrix}
        -0.8 & -0.9 & -1.0 & -1.1
    \end{bmatrix}
\end{bmatrix}$

$y = \begin{bmatrix}
    1 & 2 & 3 & 4 \\
\end{bmatrix}$

则

$\begin{bmatrix}
    \begin{bmatrix}
        -0.0 & -0.1 & -0.2 & -0.3
    \end{bmatrix} \\
    \begin{bmatrix}
        -0.4 & -0.5 & -0.6 & -0.7
    \end{bmatrix} \\
    \begin{bmatrix}
        -0.8 & -0.9 & -1.0 & -1.1
    \end{bmatrix}
\end{bmatrix} \centerdot \begin{bmatrix}
    1 & 2 & 3 & 4
\end{bmatrix} \Rightarrow \begin{bmatrix}
    \begin{bmatrix}
        -0.0 & -0.1 & -0.2 & -0.3
    \end{bmatrix} \\
    \begin{bmatrix}
        -0.4 & -0.5 & -0.6 & -0.7
    \end{bmatrix} \\
    \begin{bmatrix}
        -0.8 & -0.9 & -1.0 & -1.1
    \end{bmatrix}
\end{bmatrix} \centerdot \begin{bmatrix}
    \begin{bmatrix}
        1 & 2 & 3 & 4
    \end{bmatrix} \\
    \begin{bmatrix}
        1 & 2 & 3 & 4
    \end{bmatrix} \\
    \begin{bmatrix}
        1 & 2 & 3 & 4
    \end{bmatrix}
\end{bmatrix}
$

可以看到, 两个矩阵如果维度不同 (即矩阵的列数量不同), 但现有列的元素数一致, 则运算时, 会将列较少的矩阵进行扩展, 使其和另一个矩阵的列数一致, 扩展的方法为: 将矩阵的最后一列进行复制, 复制两个矩阵列相差数量次

推广到多维数组, 则数组广播的规则为:

- 如果两个数组的维度 (dim) 相同, 但形状在某个维度上不匹配, 且其中一个维度长度为 `1` (例如两个数组的 dim 都为 `3`, 形状分别为 `(1, 3, 4)` 和 `(2, 3, 4)`)
- 如果两个数组的维度 (dim) 不同, 则会为维度较少的数组最前面插入一个长度为 `1` 的维度 (例如两个数组的 dim 各为 `2` 和 `3`, shape 各为 `(3, 4)` 和 `(2, 3, 4)`, 使前者的 shape 变为 `(1, 3, 4)`)

如果集合满足上述两条规则, 则会在长度为 `1` 的维度上进行复制, 使两个数组的形状相同 (例如将数组从 shape `(1, 3, 4)` 通过复制变为 `(2, 3, 4)`)

对于不满足上述规则的数组, 则无法进行广播, 在运算时会抛出 `ValueError` 异常

## 2. 二维数组的广播

### 2.1. 二维数组间广播

对于两个数组 `a` 与 `b`:

- 数组 `a` 的 shape 为 `(1, 4)`
- 数组 `b` 的 shape 为 `(2, 4)`

故 `a`, `b` 两个数组的维度相同, 且 `a` 数组第一个维度的长度为 `1`, 则通过广播机制, 将 `a` 数组沿长度为 `1` 的维度进行复制, 使其 shape 变为 `(2, 4)`

$
a = \begin{bmatrix}
    \begin{bmatrix}
        1 & 2 & 3 & 4
    \end{bmatrix}
\end{bmatrix} \Rightarrow \begin{bmatrix}
    \begin{bmatrix}
        1 & 2 & 3 & 4
    \end{bmatrix} \\
    \begin{bmatrix}
        1 & 2 & 3 & 4
    \end{bmatrix}
\end{bmatrix}
$

广播后, `a` + `b` 结果如下:

$\begin{bmatrix}
    \begin{bmatrix}
        1 & 2 & 3 & 4
    \end{bmatrix} \\
    \begin{bmatrix}
        1 & 2 & 3 & 4
    \end{bmatrix}
\end{bmatrix} + \begin{bmatrix}
    \begin{bmatrix}
        5 & 6 & 7 & 8
    \end{bmatrix} \\
    \begin{bmatrix}
        9 & 10 & 11 & 12
    \end{bmatrix} \\
\end{bmatrix} = \begin{bmatrix}
    \begin{bmatrix}
        6 & 8 & 10 & 12
    \end{bmatrix} \\
    \begin{bmatrix}
        10 & 12 & 14 & 16
    \end{bmatrix} \\
\end{bmatrix}$

In [2]:
# 数组 `a` 的 shape 为 `(1, 4)` (二维数组), 数组 `b` 的 shape 为 `(2, 4)` (二维数组), 两个数组维度相同
# 1. 将数组 `a` 沿 shape 为 `1` 的维度进行复制, 使其 shape 为 `(2, 4)`, 即 `a` 变为 `[[1, 2, 3, 4], [1, 2, 3, 4]]`
# 3. 此时 `a`, `b` 数组的 shape 相同, 可以将 `a` 和 `b` 相加
a = np.array([[1, 2, 3, 4]])
b = np.array([[5, 6, 7, 8], [9, 10, 11, 12]])
aprint(
    "数组相加:",
    {
        "a": a,
        "b": b,
        "a + b": a + b,
    },
)

数组相加:
> a:
[[1 2 3 4]], shape=(1, 4)
> b:
[[ 5  6  7  8]
 [ 9 10 11 12]], shape=(2, 4)
> a + b:
[[ 6  8 10 12]
 [10 12 14 16]], shape=(2, 4)


对于两个数组 `a` 与 `b`:

- 数组 `a` 的 shape 为 `(2, 1)`
- 数组 `b` 的 shape 为 `(2, 4)`

故 `a`, `b` 两个数组的维度相同, 且数组 `a` 具备长度为 `1` 的维度, 则将 `a` 数组沿长度为 `1` 的维度进行复制, 使其 shape 变为 `(2, 4)`

$a = \begin{bmatrix}
    \begin{bmatrix}
        10
    \end{bmatrix} \\
    \begin{bmatrix}
        11
    \end{bmatrix}
\end{bmatrix} \Rightarrow \begin{bmatrix}
    \begin{bmatrix}
        10 & 10 & 10 & 10
    \end{bmatrix} \\
    \begin{bmatrix}
        11 & 11 & 11 & 11
    \end{bmatrix}
\end{bmatrix}$

广播后, `a + b` 结果如下:

$\begin{bmatrix}
    \begin{bmatrix}
        10 & 10 & 10 & 10
    \end{bmatrix} \\
    \begin{bmatrix}
        11 & 11 & 11 & 11
    \end{bmatrix}
\end{bmatrix} + \begin{bmatrix}
    \begin{bmatrix}
        5 & 6 & 7 & 8
    \end{bmatrix} \\
    \begin{bmatrix}
        9 & 10 & 11 & 12
    \end{bmatrix}
\end{bmatrix} = \begin{bmatrix}
    \begin{bmatrix}
        15 & 16 & 17 & 18
    \end{bmatrix} \\
    \begin{bmatrix}
        20 & 21 & 22 & 23
    \end{bmatrix}
\end{bmatrix}$

In [3]:
# 数组 `a` 的 shape 为 `(2, 1)` (二维数组), 数组 `b` 的 shape 为 `(2, 4)` (二维数组), 两个数组维度相同
# 1. 将数组 `a` 沿 shape 为 `1` 的维度进行复制, 使其 shape 为 `(2, 4)`, 即 `a` 变为 `[[10, 10, 10, 10], [11, 11, 11, 11]]`
# 2. 此时 `a`, `b` 数组的 shape 相同, 可以将 `a` 和 `b` 相加
a = np.array([[10], [11]])
b = np.array([[5, 6, 7, 8], [9, 10, 11, 12]])
aprint(
    "数组相加:",
    {
        "a": a,
        "b": b,
        "a + b": a + b,
    },
)

数组相加:
> a:
[[10]
 [11]], shape=(2, 1)
> b:
[[ 5  6  7  8]
 [ 9 10 11 12]], shape=(2, 4)
> a + b:
[[15 16 17 18]
 [20 21 22 23]], shape=(2, 4)


对于两个数组 `a` 与 `b`:

- 数组 `a` 的 shape 为 `(1, 1)`
- 数组 `b` 的 shape 为 `(2, 4)`

故 `a`, `b` 两个数组的维度相同, 且 `a` 数组具备两个长度为 `1` 的维度, 故将 `a` 数组沿两个长度为 `1` 的维度进行复制, 使其 shape 变为 `(2, 4)`

$a = \begin{bmatrix}
    \begin{bmatrix}
        10
    \end{bmatrix}
\end{bmatrix} \Rightarrow \begin{bmatrix}
    \begin{bmatrix}
        10 & 10 & 10 & 10
    \end{bmatrix}
\end{bmatrix} \Rightarrow \begin{bmatrix}
    \begin{bmatrix}
        10 & 10 & 10 & 10
    \end{bmatrix} \\
    \begin{bmatrix}
        10 & 10 & 10 & 10
    \end{bmatrix}
\end{bmatrix}$

广播后, `a + b` 结果如下:

$\begin{bmatrix}
    \begin{bmatrix}
        10 & 10 & 10 & 10
    \end{bmatrix} \\
    \begin{bmatrix}
        10 & 10 & 10 & 10
    \end{bmatrix}
\end{bmatrix} + \begin{bmatrix}
    \begin{bmatrix}
        5 & 6 & 7 & 8
    \end{bmatrix} \\
    \begin{bmatrix}
        9 & 10 & 11 & 12
    \end{bmatrix}
\end{bmatrix} = \begin{bmatrix}
    \begin{bmatrix}
    15 & 16 & 17 & 18
    \end{bmatrix} \\
    \begin{bmatrix}
    19 & 20 & 21 & 22
    \end{bmatrix}
\end{bmatrix}$

In [4]:
# 数组 `a` 的 shape 为 `(1, 1)` (二维数组), 数组 `b` 的 shape 为 `(2, 4)` (二维数组), 两个数组维度相同
# 1. 将数组 `a` 沿 shape 为 `1` 的维度进行复制 (两个维度), 使其 shape 为 `(2, 4)`, 即 `a` 变为 `[[10, 10, 10, 10], [10, 10, 10, 10]]`
# 2. 此时 `a`, `b` 数组的 shape 相同, 可以将 `a` 和 `b` 相加
a = np.array([[10]])
b = np.array([[5, 6, 7, 8], [9, 10, 11, 12]])
aprint(
    "数组相加:",
    {
        "a": a,
        "b": b,
        "a + b": a + b,
    },
)

数组相加:
> a:
[[10]], shape=(1, 1)
> b:
[[ 5  6  7  8]
 [ 9 10 11 12]], shape=(2, 4)
> a + b:
[[15 16 17 18]
 [19 20 21 22]], shape=(2, 4)


### 2.2. 一维和二维数组间广播

对于两个数组 `a` 与 `b`:

- 数组 `a` 的 shape 为 `(4,)`
- 数组 `b` 的 shape 为 `(2, 4)`

故 `a`, `b` 两个数组的维度不同, 则通过广播机制, 在 `a` 数组最前插入一个长度为 `1` 的维度, 使其 shape 变为 `(1, 4)`, 再沿长度为 `1` 的维度进行复制, 使其 shape 变为 `(2, 4)`

$a = \begin{bmatrix}
    1 & 2 & 3 & 4
\end{bmatrix} \Rightarrow \begin{bmatrix}
    \begin{bmatrix}
        1 & 2 & 3 & 4
    \end{bmatrix}
\end{bmatrix} \Rightarrow \begin{bmatrix}
    \begin{bmatrix}
        1 & 2 & 3 & 4
    \end{bmatrix} \\
    \begin{bmatrix}
        1 & 2 & 3 & 4
    \end{bmatrix}
\end{bmatrix}$

广播后, `a + b` 结果如下:

$\begin{bmatrix}
    \begin{bmatrix}
        1 & 2 & 3 & 4
    \end{bmatrix} \\
    \begin{bmatrix}
        1 & 2 & 3 & 4
    \end{bmatrix} \\
\end{bmatrix} + \begin{bmatrix}
    \begin{bmatrix}
        5 & 6 & 7 & 8
    \end{bmatrix} \\
    \begin{bmatrix}
        9 & 10 & 11 & 12
    \end{bmatrix} \\
\end{bmatrix} = \begin{bmatrix}
    \begin{bmatrix}
        6 & 8 & 10 & 12
    \end{bmatrix} \\
    \begin{bmatrix}
        10 & 12 & 14 & 16
    \end{bmatrix} \\
\end{bmatrix}$

In [5]:
# 数组 `a` 的 shape 为 `(4,)` (一维数组), 数组 `b` 的 shape 为 `(2, 4)` (二维数组), 两个数组维度不同
# 1. 在数组 `a` 的 shape 前面补 `1`, 使其 shape 变为 `(1, 4)`, 使其变为 `[[1, 2, 3, 4]]`
# 2. 将数组 `a` 沿 shape 为 `1` 的维度进行复制, 使其 shape 为 `(2, 4)`, 即 `a` 变为 `[[1, 2, 3, 4], [1, 2, 3, 4]]`
# 3. 此时 `a`, `b` 数组的 shape 相同, 可以将 `a` 和 `b` 相加
a = np.array([1, 2, 3, 4])
b = np.array([[5, 6, 7, 8], [9, 10, 11, 12]])
ans = a + b
aprint(
    "数组相加:",
    {
        "a": a,
        "b": b,
        "a + b": a + b,
    },
)

数组相加:
> a:
[1 2 3 4], shape=(4,)
> b:
[[ 5  6  7  8]
 [ 9 10 11 12]], shape=(2, 4)
> a + b:
[[ 6  8 10 12]
 [10 12 14 16]], shape=(2, 4)


对于两个数组 `a` 与 `b`:

- 数组 `a` 的 shape 为 `(1,)`
- 数组 `b` 的 shape 为 `(2, 4)`

故 `a`, `b` 两个数组的维度不同, 则先在 `a` 数组前面插入长度为 `1` 的维度, 使其 shape 变为 `(1, 1)`, 再沿两个长度为 `1` 的维度进行复制, 使其 shape 变为 `(2, 4)`

$a = \begin{bmatrix}
    10
\end{bmatrix} \Rightarrow \begin{bmatrix}
    \begin{bmatrix}
        10
    \end{bmatrix}
\end{bmatrix} \Rightarrow \begin{bmatrix}
    \begin{bmatrix}
        10 & 10 & 10 & 10
    \end{bmatrix}
\end{bmatrix} \Rightarrow \begin{bmatrix}
    \begin{bmatrix}
        10 & 10 & 10 & 10
    \end{bmatrix} \\
    \begin{bmatrix}
        10 & 10 & 10 & 10
    \end{bmatrix}
\end{bmatrix}$

广播后, `a + b` 结果如下:

$\begin{bmatrix}
    \begin{bmatrix}
        10 & 10 & 10 & 10
    \end{bmatrix} \\
    \begin{bmatrix}
        10 & 10 & 10 & 10
    \end{bmatrix}
\end{bmatrix} + \begin{bmatrix}
    \begin{bmatrix}
        5 & 6 & 7 & 8
    \end{bmatrix} \\
    \begin{bmatrix}
        9 & 10 & 11 & 12
    \end{bmatrix}
\end{bmatrix} = \begin{bmatrix}
    \begin{bmatrix}
        15 & 16 & 17 & 18
    \end{bmatrix} \\
    \begin{bmatrix}
        19 & 20 & 21 & 22
    \end{bmatrix}
\end{bmatrix}$

In [6]:
# 数组 `a` 的 shape 为 `(1,)` (一维数组), 数组 `b` 的 shape 为 `(2, 4)` (二维数组), 两个数组维度不同
# 1. 将数组 `a` 的 shape 前面补 `1`, 使其 shape 变为 `(1, 1)`, 即数组变为 `[[10]]`
# 2. 将数组 `a` 沿 shape 为 `1` 的维度进行复制 (两个维度), 使其 shape 为 `(2, 4)`, 即 `a` 变为 `[[10, 10, 10, 10], [10, 10, 10, 10]]`
# 3. 此时 `a`, `b` 数组的 shape 相同, 可以将 `a` 和 `b` 相加
a = np.array([10])
b = np.array([[5, 6, 7, 8], [9, 10, 11, 12]])
aprint(
    "数组相加:",
    {
        "a": a,
        "b": b,
        "a + b": a + b,
    },
)

数组相加:
> a:
[10], shape=(1,)
> b:
[[ 5  6  7  8]
 [ 9 10 11 12]], shape=(2, 4)
> a + b:
[[15 16 17 18]
 [19 20 21 22]], shape=(2, 4)


### 2.3. 无法广播的情况

对于两个数组 `a` 与 `b`:

- 数组 `a` 的 shape 为 `(2, 3)`
- 数组 `b` 的 shape 为 `(2, 4)`

两个数组的维度相同, 但形状不同, 且形状中没有长度为 `1` 的维度, 则无法进行广播, 抛出异常: `ValueError` 异常

In [7]:
a = np.array([[1, 2, 3], [4, 5, 6]])
b = np.array([[5, 6, 7, 8], [9, 10, 11, 12]])
try:
    a + b  # type: ignore
except ValueError as e:
    aprint("两个数组分别为", {"a": a, "b": b})
    print(f"{co.Fore.RED}- 数组相加失败: {e}{co.Fore.RESET}")

两个数组分别为
> a:
[[1 2 3]
 [4 5 6]], shape=(2, 3)
> b:
[[ 5  6  7  8]
 [ 9 10 11 12]], shape=(2, 4)
- 数组相加失败: operands could not be broadcast together with shapes (2,3) (2,4) 


对于两个数组 `a` 与 `b`:

- 数组 `a` 的 shape 为 `(3，)`
- 数组 `b` 的 shape 为 `(2, 4)`

两个数组的维度相同, 但形状不同, 且形状中没有长度为 `1` 的维度, 则无法进行广播, 抛出异常: `ValueError` 异常

In [8]:
a = np.array([1, 2, 3])
b = np.array([[5, 6, 7, 8], [9, 10, 11, 12]])
try:
    a + b  # type: ignore
except ValueError as e:
    aprint("两个数组分别为", {"a": a, "b": b})
    print(f"{co.Fore.RED}- 数组相加失败: {e}{co.Fore.RESET}")

两个数组分别为
> a:
[1 2 3], shape=(3,)
> b:
[[ 5  6  7  8]
 [ 9 10 11 12]], shape=(2, 4)
- 数组相加失败: operands could not be broadcast together with shapes (3,) (2,4) 


### 2.4. 二维数组广播总结

综上, 对于维度最高位二维的两个数组 `a` 和 `b` 来说, 其中一个数组如果为 `2` 维, 则另一个数组可以为的维度可以为 `1` 或 `2`

- 如果两个数组的 shape 相同, 则无需广播, 可以直接进行运算
- 如果两个数组的 shape 不同, 则 shape 中必须有一个长度为 `1` 的维度, 或两个维度长度都为 `1`, 该数组会沿着长度为 `1` 的维度进行复制, 使两个数组的 shape 相同
- 如果两个数组的 shape 不同, 且 shape 中没有为长度 `1` 的维度, 则无法进行广播, 会抛出 `ValueError`

## 3. 多维数组广播

上述二维数组的广播可以推广到多维数组

对于两个数组 `a` 与 `b`:

- 数组 `a` 的 shape 为 `(1, 3, 4)`
- 数组 `b` 的 shape 为 `(2, 3, 4)`

故 `a`, `b` 两个数组的维度相同, 且数组 `a` 具备长度为 `1` 的维度, 则将 `a` 数组沿长度为 `1` 的维度进行复制, 使其 shape 变为 `(2, 3, 4)`

$a = \begin{bmatrix}
    \begin{bmatrix}
        \begin{bmatrix}
            1 & 2 & 3 & 4
        \end{bmatrix} \\
        \begin{bmatrix}
            5 & 6 & 7 & 8
        \end{bmatrix} \\
        \begin{bmatrix}
            9 & 10 & 11 & 12
        \end{bmatrix}
    \end{bmatrix}
\end{bmatrix} \Rightarrow \begin{bmatrix}
    \begin{bmatrix}
        \begin{bmatrix}
            1 & 2 & 3 & 4
        \end{bmatrix} \\
        \begin{bmatrix}
            5 & 6 & 7 & 8
        \end{bmatrix} \\
        \begin{bmatrix}
            9 & 10 & 11 & 12
        \end{bmatrix}
    \end{bmatrix} \\
    \begin{bmatrix}
        \begin{bmatrix}
            1 & 2 & 3 & 4
        \end{bmatrix} \\
        \begin{bmatrix}
            5 & 6 & 7 & 8
        \end{bmatrix} \\
        \begin{bmatrix}
            9 & 10 & 11 & 12
        \end{bmatrix}
    \end{bmatrix}
\end{bmatrix}$

广播后, `a + b` 结果如下:

$\begin{bmatrix}
    \begin{bmatrix}
        \begin{bmatrix}
            1 & 2 & 3 & 4
        \end{bmatrix} \\
        \begin{bmatrix}
            5 & 6 & 7 & 8
        \end{bmatrix} \\
        \begin{bmatrix}
            9 & 10 & 11 & 12
        \end{bmatrix}
    \end{bmatrix} \\
    \begin{bmatrix}
        \begin{bmatrix}
            1 & 2 & 3 & 4
        \end{bmatrix} \\
        \begin{bmatrix}
            5 & 6 & 7 & 8
        \end{bmatrix} \\
        \begin{bmatrix}
            9 & 10 & 11 & 12
        \end{bmatrix}
    \end{bmatrix}
\end{bmatrix} + \begin{bmatrix}
    \begin{bmatrix}
        \begin{bmatrix}
            100 & 101 & 102 & 103
        \end{bmatrix} \\
        \begin{bmatrix}
            104 & 105 & 106 & 107
        \end{bmatrix} \\
        \begin{bmatrix}
            108 & 109 & 110 & 111
        \end{bmatrix}
    \end{bmatrix} \\
    \begin{bmatrix}
        \begin{bmatrix}
            112 & 113 & 114 & 115
        \end{bmatrix} \\
        \begin{bmatrix}
            116 & 117 & 118 & 119
        \end{bmatrix} \\
        \begin{bmatrix}
            120 & 121 & 122 & 123
        \end{bmatrix}
    \end{bmatrix}
\end{bmatrix} = \begin{bmatrix}
    \begin{bmatrix}
        \begin{bmatrix}
            101 & 103 & 105 & 107
        \end{bmatrix} \\
        \begin{bmatrix}
            109 & 111 & 113 & 115
        \end{bmatrix} \\
        \begin{bmatrix}
            117 & 119 & 121 & 123
        \end{bmatrix}
    \end{bmatrix} \\
    \begin{bmatrix}
        \begin{bmatrix}
            113 & 115 & 117 & 119
        \end{bmatrix} \\
        \begin{bmatrix}
            121 & 123 & 125 & 127
        \end{bmatrix} \\
        \begin{bmatrix}
            129 & 131 & 133 & 135
        \end{bmatrix}
    \end{bmatrix}
\end{bmatrix}$

In [9]:
a = arange_by_shape((1, 3, 4), 1)
b = arange_by_shape((2, 3, 4), 100)
aprint(
    "数组相加:",
    {
        "a": a,
        "b": b,
        "a + b": a + b,
    },
)

数组相加:
> a:
[[[ 1  2  3  4]
  [ 5  6  7  8]
  [ 9 10 11 12]]], shape=(1, 3, 4)
> b:
[[[100 101 102 103]
  [104 105 106 107]
  [108 109 110 111]]

 [[112 113 114 115]
  [116 117 118 119]
  [120 121 122 123]]], shape=(2, 3, 4)
> a + b:
[[[101 103 105 107]
  [109 111 113 115]
  [117 119 121 123]]

 [[113 115 117 119]
  [121 123 125 127]
  [129 131 133 135]]], shape=(2, 3, 4)


对于两个数组 `a` 与 `b`:

- 数组 `a` 的 shape 为 `(2, 1, 4)`
- 数组 `b` 的 shape 为 `(2, 3, 4)`

故 `a`, `b` 两个数组的维度相同, 且数组 `a` 具备长度为 `1` 的维度, 则将 `a` 数组沿长度为 `1` 的维度进行复制, 使其 shape 变为 `(2, 3, 4)`

$a = \begin{bmatrix}
    \begin{bmatrix}
        \begin{bmatrix}
            1 & 2 & 3 & 4
        \end{bmatrix}
    \end{bmatrix} \\
    \begin{bmatrix}
        \begin{bmatrix}
            5 & 6 & 7 & 8
        \end{bmatrix}
    \end{bmatrix}
\end{bmatrix} \Rightarrow \begin{bmatrix}
    \begin{bmatrix}
        \begin{bmatrix}
            1 & 2 & 3 & 4
        \end{bmatrix} \\
        \begin{bmatrix}
            1 & 2 & 3 & 4
        \end{bmatrix} \\
        \begin{bmatrix}
            1 & 2 & 3 & 4
        \end{bmatrix}
    \end{bmatrix} \\
    \begin{bmatrix}
        \begin{bmatrix}
            5 & 6 & 7 & 8
        \end{bmatrix} \\
        \begin{bmatrix}
            5 & 6 & 7 & 8
        \end{bmatrix} \\
        \begin{bmatrix}
            5 & 6 & 7 & 8
        \end{bmatrix}
    \end{bmatrix}
\end{bmatrix}$

广播后, `a + b` 结果如下:

$\begin{bmatrix}
    \begin{bmatrix}
        \begin{bmatrix}
            1 & 2 & 3 & 4
        \end{bmatrix} \\
        \begin{bmatrix}
            1 & 2 & 3 & 4
        \end{bmatrix} \\
        \begin{bmatrix}
            1 & 2 & 3 & 4
        \end{bmatrix}
    \end{bmatrix} \\
    \begin{bmatrix}
        \begin{bmatrix}
            5 & 6 & 7 & 8
        \end{bmatrix} \\
        \begin{bmatrix}
            5 & 6 & 7 & 8
        \end{bmatrix} \\
        \begin{bmatrix}
            5 & 6 & 7 & 8
        \end{bmatrix}
    \end{bmatrix}
\end{bmatrix} + \begin{bmatrix}
    \begin{bmatrix}
        \begin{bmatrix}
            100 & 101 & 102 & 103
        \end{bmatrix} \\
        \begin{bmatrix}
            104 & 105 & 106 & 107
        \end{bmatrix} \\
        \begin{bmatrix}
            108 & 109 & 110 & 111
        \end{bmatrix}
    \end{bmatrix} \\
    \begin{bmatrix}
        \begin{bmatrix}
            112 & 113 & 114 & 115
        \end{bmatrix} \\
        \begin{bmatrix}
            116 & 117 & 118 & 119
        \end{bmatrix} \\
        \begin{bmatrix}
            120 & 121 & 122 & 123
        \end{bmatrix}
    \end{bmatrix}
\end{bmatrix} = \begin{bmatrix}
    \begin{bmatrix}
        \begin{bmatrix}
            101 & 103 & 105 & 107
        \end{bmatrix} \\
        \begin{bmatrix}
            105 & 107 & 109 & 111
        \end{bmatrix} \\
        \begin{bmatrix}
            109 & 111 & 113 & 115
        \end{bmatrix}
    \end{bmatrix} \\
    \begin{bmatrix}
        \begin{bmatrix}
            117 & 119 & 121 & 123
        \end{bmatrix} \\
        \begin{bmatrix}
            121 & 123 & 125 & 127
        \end{bmatrix} \\
        \begin{bmatrix}
            125 & 127 & 129 & 131
        \end{bmatrix}
    \end{bmatrix}
\end{bmatrix}$

In [10]:
a = arange_by_shape((2, 1, 4), 1)
b = arange_by_shape((2, 3, 4), 100)
aprint(
    "数组相加:",
    {
        "a": a,
        "b": b,
        "a + b": a + b,
    },
)

数组相加:
> a:
[[[1 2 3 4]]

 [[5 6 7 8]]], shape=(2, 1, 4)
> b:
[[[100 101 102 103]
  [104 105 106 107]
  [108 109 110 111]]

 [[112 113 114 115]
  [116 117 118 119]
  [120 121 122 123]]], shape=(2, 3, 4)
> a + b:
[[[101 103 105 107]
  [105 107 109 111]
  [109 111 113 115]]

 [[117 119 121 123]
  [121 123 125 127]
  [125 127 129 131]]], shape=(2, 3, 4)


对于两个数组 `a` 与 `b`:

- 数组 `a` 的 shape 为 `(2, 3, 1)`
- 数组 `b` 的 shape 为 `(2, 3, 4)`

故 `a`, `b` 两个数组的维度相同, 且数组 `a` 具备长度为 `1` 的维度, 则将 `a` 数组沿长度为 `1` 的维度进行复制, 使其 shape 变为 `(2, 3, 4)`

$a = \begin{bmatrix}
    \begin{bmatrix}
        \begin{bmatrix}
            1
        \end{bmatrix} \\
        \begin{bmatrix}
            2
        \end{bmatrix} \\
        \begin{bmatrix}
            3
        \end{bmatrix}
    \end{bmatrix} \\
    \begin{bmatrix}
        \begin{bmatrix}
            4
        \end{bmatrix} \\
        \begin{bmatrix}
            5
        \end{bmatrix} \\
        \begin{bmatrix}
            6
        \end{bmatrix}
    \end{bmatrix}
\end{bmatrix} \Rightarrow \begin{bmatrix}
    \begin{bmatrix}
        \begin{bmatrix}
            1 & 1 & 1 & 1
        \end{bmatrix} \\
        \begin{bmatrix}
            2 & 2 & 2 & 2
        \end{bmatrix} \\
        \begin{bmatrix}
            3 & 3 & 3 & 3
        \end{bmatrix}
    \end{bmatrix} \\
    \begin{bmatrix}
        \begin{bmatrix}
            4 & 4 & 4 & 4
        \end{bmatrix} \\
        \begin{bmatrix}
            5 & 5 & 5 & 5
        \end{bmatrix} \\
        \begin{bmatrix}
            6 & 6 & 6 & 6
        \end{bmatrix}
    \end{bmatrix}
\end{bmatrix}$

广播后, `a + b` 结果如下:

$\begin{bmatrix}
    \begin{bmatrix}
        \begin{bmatrix}
            1 & 1 & 1 & 1
        \end{bmatrix} \\
        \begin{bmatrix}
            2 & 2 & 2 & 2
        \end{bmatrix} \\
        \begin{bmatrix}
            3 & 3 & 3 & 3
        \end{bmatrix}
    \end{bmatrix} \\
    \begin{bmatrix}
        \begin{bmatrix}
            4 & 4 & 4 & 4
        \end{bmatrix} \\
        \begin{bmatrix}
            5 & 5 & 5 & 5
        \end{bmatrix} \\
        \begin{bmatrix}
            6 & 6 & 6 & 6
        \end{bmatrix}
    \end{bmatrix}
\end{bmatrix} + \begin{bmatrix}
    \begin{bmatrix}
        \begin{bmatrix}
            100 & 101 & 102 & 103
        \end{bmatrix} \\
        \begin{bmatrix}
            104 & 105 & 106 & 107
        \end{bmatrix} \\
        \begin{bmatrix}
            108 & 109 & 110 & 111
        \end{bmatrix}
    \end{bmatrix} \\
    \begin{bmatrix}
        \begin{bmatrix}
            112 & 113 & 114 & 115
        \end{bmatrix} \\
        \begin{bmatrix}
            116 & 117 & 118 & 119
        \end{bmatrix} \\
        \begin{bmatrix}
            120 & 121 & 122 & 123
        \end{bmatrix}
    \end{bmatrix}
\end{bmatrix} = \begin{bmatrix}
    \begin{bmatrix}
        \begin{bmatrix}
            101 & 102 & 103 & 104
        \end{bmatrix} \\
        \begin{bmatrix}
            106 & 107 & 108 & 109
        \end{bmatrix} \\
        \begin{bmatrix}
            111 & 112 & 113 & 114
        \end{bmatrix}
    \end{bmatrix} \\
    \begin{bmatrix}
        \begin{bmatrix}
            116 & 117 & 118 & 119
        \end{bmatrix} \\
        \begin{bmatrix}
            121 & 122 & 123 & 124
        \end{bmatrix} \\
        \begin{bmatrix}
            126 & 127 & 128 & 129
        \end{bmatrix}
    \end{bmatrix}
\end{bmatrix}$

In [11]:
a = arange_by_shape((2, 3, 1), 1)
b = arange_by_shape((2, 3, 4), 100)
aprint(
    "数组相加:",
    {
        "a": a,
        "b": b,
        "a + b": a + b,
    },
)

数组相加:
> a:
[[[1]
  [2]
  [3]]

 [[4]
  [5]
  [6]]], shape=(2, 3, 1)
> b:
[[[100 101 102 103]
  [104 105 106 107]
  [108 109 110 111]]

 [[112 113 114 115]
  [116 117 118 119]
  [120 121 122 123]]], shape=(2, 3, 4)
> a + b:
[[[101 102 103 104]
  [106 107 108 109]
  [111 112 113 114]]

 [[116 117 118 119]
  [121 122 123 124]
  [126 127 128 129]]], shape=(2, 3, 4)


对于两个数组 `a` 与 `b`:

- 数组 `a` 的 shape 为 `(3, 4)`
- 数组 `b` 的 shape 为 `(2, 3, 4)`

故 `a`, `b` 两个数组的维度不, 在数组 `a` 前插入长度为 `1` 的维度, 使其 shape 变为 `(1, 3, 4)`, 再将 `a` 数组沿长度为 `1` 的维度进行复制, 使其 shape 变为 `(2, 3, 4)`

$a = \begin{bmatrix}
    \begin{bmatrix}
        1 & 2 & 3 & 4
    \end{bmatrix} \\
    \begin{bmatrix}
        5 & 6 & 7 & 8
    \end{bmatrix} \\
    \begin{bmatrix}
        9 & 10 & 11 & 12
    \end{bmatrix}
\end{bmatrix} \Rightarrow \begin{bmatrix}
    \begin{bmatrix}
        \begin{bmatrix}
            1 & 2 & 3 & 4
        \end{bmatrix} \\
        \begin{bmatrix}
            5 & 6 & 7 & 8
        \end{bmatrix} \\
        \begin{bmatrix}
            9 & 10 & 11 & 12
        \end{bmatrix}
    \end{bmatrix}
\end{bmatrix} \Rightarrow \begin{bmatrix}
    \begin{bmatrix}
        \begin{bmatrix}
            1 & 2 & 3 & 4
        \end{bmatrix} \\
        \begin{bmatrix}
            5 & 6 & 7 & 8
        \end{bmatrix} \\
        \begin{bmatrix}
            9 & 10 & 11 & 12
        \end{bmatrix}
    \end{bmatrix} \\
    \begin{bmatrix}
        \begin{bmatrix}
            1 & 2 & 3 & 4
        \end{bmatrix} \\
        \begin{bmatrix}
            5 & 6 & 7 & 8
        \end{bmatrix} \\
        \begin{bmatrix}
            9 & 10 & 11 & 12
        \end{bmatrix}
    \end{bmatrix}
\end{bmatrix}$

广播后, `a + b` 结果如下:

$\begin{bmatrix}
    \begin{bmatrix}
        \begin{bmatrix}
            1 & 2 & 3 & 4
        \end{bmatrix} \\
        \begin{bmatrix}
            5 & 6 & 7 & 8
        \end{bmatrix} \\
        \begin{bmatrix}
            9 & 10 & 11 & 12
        \end{bmatrix}
    \end{bmatrix} \\
    \begin{bmatrix}
        \begin{bmatrix}
            1 & 2 & 3 & 4
        \end{bmatrix} \\
        \begin{bmatrix}
            5 & 6 & 7 & 8
        \end{bmatrix} \\
        \begin{bmatrix}
            9 & 10 & 11 & 12
        \end{bmatrix}
    \end{bmatrix}
\end{bmatrix} + \begin{bmatrix}
    \begin{bmatrix}
        \begin{bmatrix}
            100 & 101 & 102 & 103
        \end{bmatrix} \\
        \begin{bmatrix}
            104 & 105 & 106 & 107
        \end{bmatrix} \\
        \begin{bmatrix}
            108 & 109 & 110 & 111
        \end{bmatrix}
    \end{bmatrix} \\
    \begin{bmatrix}
        \begin{bmatrix}
            112 & 113 & 114 & 115
        \end{bmatrix} \\
        \begin{bmatrix}
            116 & 117 & 118 & 119
        \end{bmatrix} \\
        \begin{bmatrix}
            120 & 121 & 122 & 123
        \end{bmatrix}
    \end{bmatrix}
\end{bmatrix} = \begin{bmatrix}
    \begin{bmatrix}
        \begin{bmatrix}
            101 & 103 & 105 & 107
        \end{bmatrix} \\
        \begin{bmatrix}
            109 & 111 & 113 & 115
        \end{bmatrix} \\
        \begin{bmatrix}
            117 & 119 & 121 & 123
        \end{bmatrix}
    \end{bmatrix} \\
    \begin{bmatrix}
        \begin{bmatrix}
            113 & 115 & 117 & 119
        \end{bmatrix} \\
        \begin{bmatrix}
            121 & 122 & 123 & 124
        \end{bmatrix} \\
        \begin{bmatrix}
            129 & 131 & 133 & 135
        \end{bmatrix}
    \end{bmatrix}
\end{bmatrix}$

In [12]:
a = arange_by_shape((2, 3, 1), 1)
b = arange_by_shape((2, 3, 4), 100)
aprint(
    "数组相加:",
    {
        "a": a,
        "b": b,
        "a + b": a + b,
    },
)

数组相加:
> a:
[[[1]
  [2]
  [3]]

 [[4]
  [5]
  [6]]], shape=(2, 3, 1)
> b:
[[[100 101 102 103]
  [104 105 106 107]
  [108 109 110 111]]

 [[112 113 114 115]
  [116 117 118 119]
  [120 121 122 123]]], shape=(2, 3, 4)
> a + b:
[[[101 102 103 104]
  [106 107 108 109]
  [111 112 113 114]]

 [[116 117 118 119]
  [121 122 123 124]
  [126 127 128 129]]], shape=(2, 3, 4)


对于两个数组 `a` 与 `b`:

- 数组 `a` 的 shape 为 `(4，)`
- 数组 `b` 的 shape 为 `(2, 3, 4)`

故 `a`, `b` 两个数组的维度不同, 在 `a` 数组前插入两个长度为 `1` 的维度, 使其 shape 变为 `(1, 1, 4)`, 再将 `a` 数组沿两个长度为 `1` 的维度进行复制, 使其 shape 变为 `(2, 3, 4)`

$a = \begin{bmatrix}
    1 & 2 & 3 & 4
\end{bmatrix} \Rightarrow \begin{bmatrix}
    \begin{bmatrix}
        1 & 2 & 3 & 4
    \end{bmatrix}
\end{bmatrix} \Rightarrow \begin{bmatrix}
    \begin{bmatrix}
        \begin{bmatrix}
            1 & 2 & 3 & 4
        \end{bmatrix}
    \end{bmatrix}
\end{bmatrix} \Rightarrow \begin{bmatrix}
    \begin{bmatrix}
        \begin{bmatrix}
            1 & 2 & 3 & 4
        \end{bmatrix} \\
        \begin{bmatrix}
            1 & 2 & 3 & 4
        \end{bmatrix} \\
        \begin{bmatrix}
            1 & 2 & 3 & 4
        \end{bmatrix}
    \end{bmatrix}
\end{bmatrix} \Rightarrow \begin{bmatrix}
    \begin{bmatrix}
        \begin{bmatrix}
            1 & 2 & 3 & 4
        \end{bmatrix} \\
        \begin{bmatrix}
            1 & 2 & 3 & 4
        \end{bmatrix} \\
        \begin{bmatrix}
            1 & 2 & 3 & 4
        \end{bmatrix}
    \end{bmatrix} \\
    \begin{bmatrix}
        \begin{bmatrix}
            1 & 2 & 3 & 4
        \end{bmatrix} \\
        \begin{bmatrix}
            1 & 2 & 3 & 4
        \end{bmatrix} \\
        \begin{bmatrix}
            1 & 2 & 3 & 4
        \end{bmatrix}
    \end{bmatrix}
\end{bmatrix}$

广播后, `a + b` 结果如下:

$\begin{bmatrix}
    \begin{bmatrix}
        \begin{bmatrix}
            1 & 2 & 3 & 4
        \end{bmatrix} \\
        \begin{bmatrix}
            1 & 2 & 3 & 4
        \end{bmatrix} \\
        \begin{bmatrix}
            1 & 2 & 3 & 4
        \end{bmatrix}
    \end{bmatrix} \\
    \begin{bmatrix}
        \begin{bmatrix}
            1 & 2 & 3 & 4
        \end{bmatrix} \\
        \begin{bmatrix}
            1 & 2 & 3 & 4
        \end{bmatrix} \\
        \begin{bmatrix}
            1 & 2 & 3 & 4
        \end{bmatrix}
    \end{bmatrix}
\end{bmatrix} + \begin{bmatrix}
    \begin{bmatrix}
        \begin{bmatrix}
            100 & 101 & 102 & 103
        \end{bmatrix} \\
        \begin{bmatrix}
            104 & 105 & 106 & 107
        \end{bmatrix} \\
        \begin{bmatrix}
            108 & 109 & 110 & 111
        \end{bmatrix}
    \end{bmatrix} \\
    \begin{bmatrix}
        \begin{bmatrix}
            112 & 113 & 114 & 115
        \end{bmatrix} \\
        \begin{bmatrix}
            116 & 117 & 118 & 119
        \end{bmatrix} \\
        \begin{bmatrix}
            120 & 121 & 122 & 123
        \end{bmatrix}
    \end{bmatrix}
\end{bmatrix} = \begin{bmatrix}
    \begin{bmatrix}
        \begin{bmatrix}
            101 & 103 & 105 & 107
        \end{bmatrix} \\
        \begin{bmatrix}
            105 & 107 & 109 & 111
        \end{bmatrix} \\
        \begin{bmatrix}
            109 & 111 & 113 & 115
        \end{bmatrix}
    \end{bmatrix} \\
    \begin{bmatrix}
        \begin{bmatrix}
            113 & 115 & 117 & 119
        \end{bmatrix} \\
        \begin{bmatrix}
            117 & 119 & 121 & 123
        \end{bmatrix} \\
        \begin{bmatrix}
            121 & 123 & 125 & 127
        \end{bmatrix}
    \end{bmatrix}
\end{bmatrix}$

In [13]:
a = arange_by_shape((2, 3, 1), 1)
b = arange_by_shape((2, 3, 4), 100)
aprint(
    "数组相加:",
    {
        "a": a,
        "b": b,
        "a + b": a + b,
    },
)

数组相加:
> a:
[[[1]
  [2]
  [3]]

 [[4]
  [5]
  [6]]], shape=(2, 3, 1)
> b:
[[[100 101 102 103]
  [104 105 106 107]
  [108 109 110 111]]

 [[112 113 114 115]
  [116 117 118 119]
  [120 121 122 123]]], shape=(2, 3, 4)
> a + b:
[[[101 102 103 104]
  [106 107 108 109]
  [111 112 113 114]]

 [[116 117 118 119]
  [121 122 123 124]
  [126 127 128 129]]], shape=(2, 3, 4)


对于两个数组 `a` 与 `b`:

- 数组 `a` 的 shape 为 `(1, 1, 4)`
- 数组 `b` 的 shape 为 `(2, 3, 4)`

故 `a`, `b` 两个数组的维度相同, 且具备两个长度为 `1` 的维度, 则将 `a` 数组沿两个长度为 `1` 的维度进行复制, 使其 shape 变为 `(2, 3, 4)`

$a = \begin{bmatrix}
    \begin{bmatrix}
        \begin{bmatrix}
            1 & 2 & 3 & 4
        \end{bmatrix}
    \end{bmatrix}
\end{bmatrix} \Rightarrow \begin{bmatrix}
    \begin{bmatrix}
        \begin{bmatrix}
            1 & 2 & 3 & 4
        \end{bmatrix} \\
        \begin{bmatrix}
            1 & 2 & 3 & 4
        \end{bmatrix} \\
        \begin{bmatrix}
            1 & 2 & 3 & 4
        \end{bmatrix}
    \end{bmatrix}
\end{bmatrix} \Rightarrow \begin{bmatrix}
    \begin{bmatrix}
        \begin{bmatrix}
            1 & 2 & 3 & 4
        \end{bmatrix} \\
        \begin{bmatrix}
            1 & 2 & 3 & 4
        \end{bmatrix} \\
        \begin{bmatrix}
            1 & 2 & 3 & 4
        \end{bmatrix}
    \end{bmatrix} \\
    \begin{bmatrix}
        \begin{bmatrix}
            1 & 2 & 3 & 4
        \end{bmatrix} \\
        \begin{bmatrix}
            1 & 2 & 3 & 4
        \end{bmatrix} \\
        \begin{bmatrix}
            1 & 2 & 3 & 4
        \end{bmatrix}
    \end{bmatrix}
\end{bmatrix}$

广播后, `a + b` 结果如下:

$\begin{bmatrix}
    \begin{bmatrix}
        \begin{bmatrix}
            1 & 2 & 3 & 4
        \end{bmatrix} \\
        \begin{bmatrix}
            1 & 2 & 3 & 4
        \end{bmatrix} \\
        \begin{bmatrix}
            1 & 2 & 3 & 4
        \end{bmatrix}
    \end{bmatrix} \\
    \begin{bmatrix}
        \begin{bmatrix}
            1 & 2 & 3 & 4
        \end{bmatrix} \\
        \begin{bmatrix}
            1 & 2 & 3 & 4
        \end{bmatrix} \\
        \begin{bmatrix}
            1 & 2 & 3 & 4
        \end{bmatrix}
    \end{bmatrix}
\end{bmatrix} + \begin{bmatrix}
    \begin{bmatrix}
        \begin{bmatrix}
            100 & 101 & 102 & 103
        \end{bmatrix} \\
        \begin{bmatrix}
            104 & 105 & 106 & 107
        \end{bmatrix} \\
        \begin{bmatrix}
            108 & 109 & 110 & 111
        \end{bmatrix}
    \end{bmatrix} \\
    \begin{bmatrix}
        \begin{bmatrix}
            112 & 113 & 114 & 115
        \end{bmatrix} \\
        \begin{bmatrix}
            116 & 117 & 118 & 119
        \end{bmatrix} \\
        \begin{bmatrix}
            120 & 121 & 122 & 123
        \end{bmatrix}
    \end{bmatrix}
\end{bmatrix} = \begin{bmatrix}
    \begin{bmatrix}
        \begin{bmatrix}
            101 & 103 & 105 & 107
        \end{bmatrix} \\
        \begin{bmatrix}
            105 & 107 & 109 & 111
        \end{bmatrix} \\
        \begin{bmatrix}
            109 & 111 & 113 & 115
        \end{bmatrix}
    \end{bmatrix} \\
    \begin{bmatrix}
        \begin{bmatrix}
            113 & 115 & 117 & 119
        \end{bmatrix} \\
        \begin{bmatrix}
            117 & 119 & 121 & 123
        \end{bmatrix} \\
        \begin{bmatrix}
            121 & 123 & 125 & 127
        \end{bmatrix}
    \end{bmatrix}
\end{bmatrix}$

In [14]:
a = arange_by_shape((2, 3, 1), 1)
b = arange_by_shape((2, 3, 4), 100)
aprint(
    "数组相加:",
    {
        "a": a,
        "b": b,
        "a + b": a + b,
    },
)

数组相加:
> a:
[[[1]
  [2]
  [3]]

 [[4]
  [5]
  [6]]], shape=(2, 3, 1)
> b:
[[[100 101 102 103]
  [104 105 106 107]
  [108 109 110 111]]

 [[112 113 114 115]
  [116 117 118 119]
  [120 121 122 123]]], shape=(2, 3, 4)
> a + b:
[[[101 102 103 104]
  [106 107 108 109]
  [111 112 113 114]]

 [[116 117 118 119]
  [121 122 123 124]
  [126 127 128 129]]], shape=(2, 3, 4)


对于两个数组 `a` 与 `b`:

- 数组 `a` 的 shape 为 `(1, 3, 1)`
- 数组 `b` 的 shape 为 `(2, 3, 4)`

故 `a`, `b` 两个数组的维度相同, 且具备两个长度为 `1` 的维度, 则将 `a` 数组沿两个长度为 `1` 的维度进行复制, 使其 shape 变为 `(2, 3, 4)`

$a = \begin{bmatrix}
    \begin{bmatrix}
        \begin{bmatrix}
            1
        \end{bmatrix} \\
        \begin{bmatrix}
            2
        \end{bmatrix} \\
        \begin{bmatrix}
            3
        \end{bmatrix}
    \end{bmatrix}
\end{bmatrix} \Rightarrow \begin{bmatrix}
    \begin{bmatrix}
        \begin{bmatrix}
            1 & 1 & 1 & 1
        \end{bmatrix} \\
        \begin{bmatrix}
            2 & 2 & 2 & 2
        \end{bmatrix} \\
        \begin{bmatrix}
            3 & 3 & 3 & 3
        \end{bmatrix}
    \end{bmatrix} \\
\end{bmatrix} \Rightarrow \begin{bmatrix}
    \begin{bmatrix}
        \begin{bmatrix}
            1 & 1 & 1 & 1
        \end{bmatrix} \\
        \begin{bmatrix}
            2 & 2 & 2 & 2
        \end{bmatrix} \\
        \begin{bmatrix}
            3 & 3 & 3 & 3
        \end{bmatrix}
    \end{bmatrix} \\
    \begin{bmatrix}
        \begin{bmatrix}
            1 & 1 & 1 & 1
        \end{bmatrix} \\
        \begin{bmatrix}
            2 & 2 & 2 & 2
        \end{bmatrix} \\
        \begin{bmatrix}
            3 & 3 & 3 & 3
        \end{bmatrix}
    \end{bmatrix}
\end{bmatrix}$

广播后, `a + b` 结果如下:

$\begin{bmatrix}
    \begin{bmatrix}
        \begin{bmatrix}
            1 & 1 & 1 & 1
        \end{bmatrix} \\
        \begin{bmatrix}
            2 & 2 & 2 & 2
        \end{bmatrix} \\
        \begin{bmatrix}
            3 & 3 & 3 & 3
        \end{bmatrix}
    \end{bmatrix} \\
    \begin{bmatrix}
        \begin{bmatrix}
            1 & 1 & 1 & 1
        \end{bmatrix} \\
        \begin{bmatrix}
            2 & 2 & 2 & 2
        \end{bmatrix} \\
        \begin{bmatrix}
            3 & 3 & 3 & 3
        \end{bmatrix}
    \end{bmatrix}
\end{bmatrix} + \begin{bmatrix}
    \begin{bmatrix}
        \begin{bmatrix}
            100 & 101 & 102 & 103
        \end{bmatrix} \\
        \begin{bmatrix}
            104 & 105 & 106 & 107
        \end{bmatrix} \\
        \begin{bmatrix}
            108 & 109 & 110 & 111
        \end{bmatrix}
    \end{bmatrix} \\
    \begin{bmatrix}
        \begin{bmatrix}
            112 & 113 & 114 & 115
        \end{bmatrix} \\
        \begin{bmatrix}
            116 & 117 & 118 & 119
        \end{bmatrix} \\
        \begin{bmatrix}
            120 & 121 & 122 & 123
        \end{bmatrix}
    \end{bmatrix}
\end{bmatrix} = \begin{bmatrix}
    \begin{bmatrix}
        \begin{bmatrix}
            101 & 102 & 103 & 104
        \end{bmatrix} \\
        \begin{bmatrix}
            106 & 107 & 108 & 109
        \end{bmatrix} \\
        \begin{bmatrix}
            111 & 112 & 113 & 114
        \end{bmatrix}
    \end{bmatrix} \\
    \begin{bmatrix}
        \begin{bmatrix}
            113 & 114 & 115 & 116
        \end{bmatrix} \\
        \begin{bmatrix}
            118 & 119 & 120 & 121
        \end{bmatrix} \\
        \begin{bmatrix}
            123 & 124 & 125 & 126
        \end{bmatrix}
    \end{bmatrix}
\end{bmatrix}$

In [15]:
a = arange_by_shape((2, 3, 1), 1)
b = arange_by_shape((2, 3, 4), 100)
aprint(
    "数组相加:",
    {
        "a": a,
        "b": b,
        "a + b": a + b,
    },
)

数组相加:
> a:
[[[1]
  [2]
  [3]]

 [[4]
  [5]
  [6]]], shape=(2, 3, 1)
> b:
[[[100 101 102 103]
  [104 105 106 107]
  [108 109 110 111]]

 [[112 113 114 115]
  [116 117 118 119]
  [120 121 122 123]]], shape=(2, 3, 4)
> a + b:
[[[101 102 103 104]
  [106 107 108 109]
  [111 112 113 114]]

 [[116 117 118 119]
  [121 122 123 124]
  [126 127 128 129]]], shape=(2, 3, 4)


对于两个数组 `a` 与 `b`:

- 数组 `a` 的 shape 为 `(1, 1, 1)`
- 数组 `b` 的 shape 为 `(2, 3, 4)`

故 `a`, `b` 两个数组的维度相同, 且具备三个长度为 `1` 的维度, 则将 `a` 数组沿三个长度为 `1` 的维度进行复制, 使其 shape 变为 `(2, 3, 4)`

$a = \begin{bmatrix}
    \begin{bmatrix}
        \begin{bmatrix}
            10
        \end{bmatrix}
    \end{bmatrix}
\end{bmatrix} \Rightarrow \begin{bmatrix}
    \begin{bmatrix}
        \begin{bmatrix}
            10 & 10 & 10 & 10
        \end{bmatrix}
    \end{bmatrix}
\end{bmatrix} \Rightarrow \begin{bmatrix}
    \begin{bmatrix}
        \begin{bmatrix}
            10 & 10 & 10 & 10
        \end{bmatrix} \\
        \begin{bmatrix}
            10 & 10 & 10 & 10
        \end{bmatrix} \\
        \begin{bmatrix}
            10 & 10 & 10 & 10
        \end{bmatrix}
    \end{bmatrix}
\end{bmatrix} \Rightarrow \begin{bmatrix}
    \begin{bmatrix}
        \begin{bmatrix}
            10 & 10 & 10 & 10
        \end{bmatrix} \\
        \begin{bmatrix}
            10 & 10 & 10 & 10
        \end{bmatrix} \\
        \begin{bmatrix}
            10 & 10 & 10 & 10
        \end{bmatrix}
    \end{bmatrix} \\
    \begin{bmatrix}
        \begin{bmatrix}
            10 & 10 & 10 & 10
        \end{bmatrix} \\
        \begin{bmatrix}
            10 & 10 & 10 & 10
        \end{bmatrix} \\
        \begin{bmatrix}
            10 & 10 & 10 & 10
        \end{bmatrix}
    \end{bmatrix}
\end{bmatrix}$

广播后, `a + b` 结果如下:

$\begin{bmatrix}
    \begin{bmatrix}
        \begin{bmatrix}
            10 & 10 & 10 & 10
        \end{bmatrix} \\
        \begin{bmatrix}
            10 & 10 & 10 & 10
        \end{bmatrix} \\
        \begin{bmatrix}
            10 & 10 & 10 & 10
        \end{bmatrix}
    \end{bmatrix} \\
    \begin{bmatrix}
        \begin{bmatrix}
            10 & 10 & 10 & 10
        \end{bmatrix} \\
        \begin{bmatrix}
            10 & 10 & 10 & 10
        \end{bmatrix} \\
        \begin{bmatrix}
            10 & 10 & 10 & 10
        \end{bmatrix}
    \end{bmatrix}
\end{bmatrix} + \begin{bmatrix}
    \begin{bmatrix}
        \begin{bmatrix}
            100 & 101 & 102 & 103
        \end{bmatrix} \\
        \begin{bmatrix}
            104 & 105 & 106 & 107
        \end{bmatrix} \\
        \begin{bmatrix}
            108 & 109 & 110 & 111
        \end{bmatrix}
    \end{bmatrix} \\
    \begin{bmatrix}
        \begin{bmatrix}
            112 & 113 & 114 & 115
        \end{bmatrix} \\
        \begin{bmatrix}
            116 & 117 & 118 & 119
        \end{bmatrix} \\
        \begin{bmatrix}
            120 & 121 & 122 & 123
        \end{bmatrix}
    \end{bmatrix}
\end{bmatrix} = \begin{bmatrix}
    \begin{bmatrix}
        \begin{bmatrix}
            110 & 111 & 112 & 113
        \end{bmatrix} \\
        \begin{bmatrix}
            114 & 115 & 116 & 117
        \end{bmatrix} \\
        \begin{bmatrix}
            118 & 119 & 120 & 121
        \end{bmatrix}
    \end{bmatrix} \\
    \begin{bmatrix}
        \begin{bmatrix}
            122 & 123 & 124 & 125
        \end{bmatrix} \\
        \begin{bmatrix}
            126 & 127 & 128 & 129
        \end{bmatrix} \\
        \begin{bmatrix}
            130 & 131 & 132 & 133
        \end{bmatrix}
    \end{bmatrix}
\end{bmatrix}$

In [16]:
a = arange_by_shape((2, 3, 1), 1)
b = arange_by_shape((2, 3, 4), 100)
aprint(
    "数组相加:",
    {
        "a": a,
        "b": b,
        "a + b": a + b,
    },
)

数组相加:
> a:
[[[1]
  [2]
  [3]]

 [[4]
  [5]
  [6]]], shape=(2, 3, 1)
> b:
[[[100 101 102 103]
  [104 105 106 107]
  [108 109 110 111]]

 [[112 113 114 115]
  [116 117 118 119]
  [120 121 122 123]]], shape=(2, 3, 4)
> a + b:
[[[101 102 103 104]
  [106 107 108 109]
  [111 112 113 114]]

 [[116 117 118 119]
  [121 122 123 124]
  [126 127 128 129]]], shape=(2, 3, 4)


对于两个数组 `a` 与 `b`:

- 数组 `a` 的 shape 为 `(1,)`
- 数组 `b` 的 shape 为 `(2, 3, 4)`

故 `a`, `b` 两个数组的维度不同, 为 `a` 数组插入两个长度为 `1` 的维度, 使其 shape 变为 `(1, 1, 1)`, 再将 `a` 数组沿三个长度为 `1` 的维度进行复制, 使其 shape 变为 `(2, 3, 4)`

$a = \begin{bmatrix}
    10
\end{bmatrix} \Rightarrow\begin{bmatrix}
    \begin{bmatrix}
        10
    \end{bmatrix}
\end{bmatrix} \Rightarrow\begin{bmatrix}
    \begin{bmatrix}
        \begin{bmatrix}
            10
        \end{bmatrix}
    \end{bmatrix}
\end{bmatrix} \Rightarrow \begin{bmatrix}
    \begin{bmatrix}
        \begin{bmatrix}
            10 & 10 & 10 & 10
        \end{bmatrix} \\
    \end{bmatrix}
\end{bmatrix} \Rightarrow \begin{bmatrix}
    \begin{bmatrix}
        \begin{bmatrix}
            10 & 10 & 10 & 10
        \end{bmatrix} \\
        \begin{bmatrix}
            10 & 10 & 10 & 10
        \end{bmatrix} \\
        \begin{bmatrix}
            10 & 10 & 10 & 10
        \end{bmatrix}
    \end{bmatrix}
\end{bmatrix} \Rightarrow \begin{bmatrix}
    \begin{bmatrix}
        \begin{bmatrix}
            10 & 10 & 10 & 10
        \end{bmatrix} \\
        \begin{bmatrix}
            10 & 10 & 10 & 10
        \end{bmatrix} \\
        \begin{bmatrix}
            10 & 10 & 10 & 10
        \end{bmatrix}
    \end{bmatrix} \\
    \begin{bmatrix}
        \begin{bmatrix}
            10 & 10 & 10 & 10
        \end{bmatrix} \\
        \begin{bmatrix}
            10 & 10 & 10 & 10
        \end{bmatrix} \\
        \begin{bmatrix}
            10 & 10 & 10 & 10
        \end{bmatrix}
    \end{bmatrix}
\end{bmatrix}$

广播后, `a + b` 结果如下:

$\begin{bmatrix}
    \begin{bmatrix}
        \begin{bmatrix}
            10 & 10 & 10 & 10
        \end{bmatrix} \\
        \begin{bmatrix}
            10 & 10 & 10 & 10
        \end{bmatrix} \\
        \begin{bmatrix}
            10 & 10 & 10 & 10
        \end{bmatrix}
    \end{bmatrix} \\
    \begin{bmatrix}
        \begin{bmatrix}
            10 & 10 & 10 & 10
        \end{bmatrix} \\
        \begin{bmatrix}
            10 & 10 & 10 & 10
        \end{bmatrix} \\
        \begin{bmatrix}
            10 & 10 & 10 & 10
        \end{bmatrix}
    \end{bmatrix}
\end{bmatrix} + \begin{bmatrix}
    \begin{bmatrix}
        \begin{bmatrix}
            100 & 101 & 102 & 103
        \end{bmatrix} \\
        \begin{bmatrix}
            104 & 105 & 106 & 107
        \end{bmatrix} \\
        \begin{bmatrix}
            108 & 109 & 110 & 111
        \end{bmatrix}
    \end{bmatrix} \\
    \begin{bmatrix}
        \begin{bmatrix}
            112 & 113 & 114 & 115
        \end{bmatrix} \\
        \begin{bmatrix}
            116 & 117 & 118 & 119
        \end{bmatrix} \\
        \begin{bmatrix}
            120 & 121 & 122 & 123
        \end{bmatrix}
    \end{bmatrix}
\end{bmatrix} = \begin{bmatrix}
    \begin{bmatrix}
        \begin{bmatrix}
            110 & 111 & 112 & 113
        \end{bmatrix} \\
        \begin{bmatrix}
            114 & 115 & 116 & 117
        \end{bmatrix} \\
        \begin{bmatrix}
            118 & 119 & 120 & 121
        \end{bmatrix}
    \end{bmatrix} \\
    \begin{bmatrix}
        \begin{bmatrix}
            122 & 123 & 124 & 125
        \end{bmatrix} \\
        \begin{bmatrix}
            126 & 127 & 128 & 129
        \end{bmatrix} \\
        \begin{bmatrix}
            130 & 131 & 132 & 133
        \end{bmatrix}
    \end{bmatrix}
\end{bmatrix}$

In [17]:
a = arange_by_shape((1,), 10)
b = arange_by_shape((2, 3, 4), 100)
aprint(
    "数组相加:",
    {
        "a": a,
        "b": b,
        "a + b": a + b,
    },
)

数组相加:
> a:
[10], shape=(1,)
> b:
[[[100 101 102 103]
  [104 105 106 107]
  [108 109 110 111]]

 [[112 113 114 115]
  [116 117 118 119]
  [120 121 122 123]]], shape=(2, 3, 4)
> a + b:
[[[110 111 112 113]
  [114 115 116 117]
  [118 119 120 121]]

 [[122 123 124 125]
  [126 127 128 129]
  [130 131 132 133]]], shape=(2, 3, 4)
